<a href="https://colab.research.google.com/github/cminy/FC2022code/blob/2204_kyo/sql_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mysql 사용을 위한 패키지 설치
!sudo apt-get install libmysqlclient-dev
!pip install mysqlclient

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmysqlclient-dev is already the newest version (5.7.37-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
# 패키지 코드를 사용할 수 있도록 패키지 import
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import MySQLdb

In [ ]:
db = MySQLdb.connect(host="0.tcp.ngrok.io", port=18518, user="root", passwd="mk246900", auth_plugin='mysql_native_password', db="world", charset="utf8")
curs = db.cursor()

In [ ]:
# Q1 : 상영시간이 60~120분 사이의 영화설명에 robot이 들어있는 영화를 상영시간 내림차순으로 정렬
# 5위에서 8위까지 영화 출력 (film_id, title, description, length)
qr = 'use sakila;'
curs.execute(qr)
qr = '''
    select film_id, title, description, length
    from film
    where (length between 60 and 120) AND (description like "%robot%")
    order by length DESC
    limit 4, 4;
''' 
an = pd.read_sql(qr, db)
an

,film_id,title,description,length
0,81,BLINDNESS GUN,A Touching Drama of a Robot And a Dentist who ...,103
1,737,ROCK INSTINCT,A Astounding Character Study of a Robot And a ...,102
2,752,RUNNER MADIGAN,A Thoughtful Documentary of a Crocodile And a ...,101
3,994,WYOMING STORM,A Awe-Inspiring Panorama of a Robot And a Boat...,100


In [ ]:
qr = 'USE world;'
curs.execute(qr)
qr = 'DESC city;'
pd.read_sql(qr, db)

,Field,Type,Null,Key,Default,Extra
0,ID,int,NO,PRI,None,auto_increment
1,Name,char(35),NO,,,
2,CountryCode,char(3),NO,MUL,,
3,District,char(20),NO,,,
4,Population,int,NO,,0,


In [ ]:
# 데이터 베이스 생성
QUERY = "CREATE DATABASE if not exists kyobo;"
curs.execute(QUERY)
QUERY = "USE kyobo;"
curs.execute(QUERY)

0

In [ ]:
QUERY = '''
CREATE TABLE if not exists user2(
      user_id INT PRIMARY KEY AUTO_INCREMENT,
      name VARCHAR(20) NOT NULL,
      age INT DEFAULT 30,
      rdate TIMESTAMP
    )
'''
curs.execute(QUERY)

0

In [ ]:
# Question : 아래와 같은 테이블이 있을 때 다음 중 틀린 설명을 고르시오.
QUERY = "USE kyobo;"
curs.execute(QUERY)
QUERY = 'DESC user2;'
pd.read_sql(QUERY, db)

,Field,Type,Null,Key,Default,Extra
0,user_id,int,NO,PRI,None,auto_increment
1,name,varchar(20),NO,,None,
2,age,int,YES,,30,
3,rdate,timestamp,YES,,None,


In [ ]:
qr = "show variables like 'character_set_database'"
pd.read_sql(qr,db) #mb4는 이모지가 추가된거

,Variable_name,Value
0,character_set_database,utf8mb3


In [ ]:
qr = '''
  alter database kyobo character set = utf8;
'''
curs.execute(qr)
qr = "show variables like 'character_set_database'"
pd.read_sql(qr,db) #mb3는 이모지 못쓰는거

,Variable_name,Value
0,character_set_database,utf8mb3


In [ ]:
qr = "ALTER TABLE user2 ADD test TEXT"
curs.execute(qr)

0

In [ ]:
qr = "alter table user2 modify test INT default 0"
curs.execute(qr)

0

In [ ]:
QUERY = 'DESC user2;'
pd.read_sql(QUERY, db)

,Field,Type,Null,Key,Default,Extra
0,user_id,int,NO,PRI,None,auto_increment
1,name,varchar(20),NO,,None,
2,age,int,YES,,30,
3,rdate,timestamp,YES,,None,
4,test,int,YES,,0,


In [ ]:
QUERY = "alter table user2 drop test;"
curs.execute(QUERY)

0

In [ ]:
QUERY = 'DESC user2;'
pd.read_sql(QUERY, db)

,Field,Type,Null,Key,Default,Extra
0,user_id,int,NO,PRI,None,auto_increment
1,name,varchar(20),NO,,None,
2,age,int,YES,,30,
3,rdate,timestamp,YES,,None,


In [ ]:
qr = '''
insert into user2 (name, age)
VALUES ('peter', 23), ('andy', 42), ('jhon', 32); 
'''
curs.execute(qr)
qr = 'select * from user2;'
pd.read_sql(qr, db)

,user_id,name,age,rdate
0,1,peter,23,None
1,2,andy,42,None
2,3,jhon,32,None


In [ ]:
qr = '''
insert into user2 (name)
VALUES ('pet'), ('sandy'), ('jane'); 
'''
curs.execute(qr)
qr = 'select * from user2;'
pd.read_sql(qr, db)

,user_id,name,age,rdate
0,1,peter,23,None
1,2,andy,42,None
2,3,jhon,32,None
3,4,pet,30,None
4,5,sandy,30,None
5,6,jane,30,None


In [ ]:
qr = '''
update user2
set age = 50, rdate = "2022-01-01" # 여기까지하면 모든 age 값이 50으로 변함
where user_id > 4
limit 1; # where 조건에 해당하는 것 중 limit까지 일부만 변경됨
'''
curs.execute(qr)
qr = 'select * from user2;'
pd.read_sql(qr, db)

,user_id,name,age,rdate
0,1,peter,23,NaT
1,2,andy,42,NaT
2,3,jhon,32,NaT
3,4,pet,30,NaT
4,5,sandy,50,2022-01-01
5,6,jane,30,NaT


In [ ]:
qr = '''
delete from user2
where user_id > 4
limit 1;
'''
curs.execute(qr)
qr = 'select * from user2;'
pd.read_sql(qr, db)

,user_id,name,age,rdate
0,1,peter,23,None
1,2,andy,42,None
2,3,jhon,32,None
3,4,pet,30,None
4,6,jane,30,None


In [ ]:
# DML : CRUD : CREATE(INSERT INTO), READ(SELECT FROM)
#              , UPDATE(UPDATE SET), DELETE(DELETE FROM)
# DDL : CRUD : CREATE(CREATE), READ(SHOW, DESC), UPDATE(ALTER), DELETE(TRUNCATE, DROP)

In [ ]:
qr = "truncate user;"
curs.execute(qr)
qr = 'select * from user;'
pd.read_sql(qr, db)

,user_id,name


In [ ]:
qr = "drop table user;"
curs.execute(qr)

0

In [ ]:
qr = "show tables;"
pd.read_sql(qr,db)

,Tables_in_kyobo
0,addr
1,user2


In [ ]:
qr = "drop database kyobo;"
curs.execute(qr)
qr = "show databases;"
pd.read_sql(qr, db)

,Database
0,information_schema
1,mysql
2,performance_schema
3,sakila
4,sys
5,testdb
6,world


In [ ]:
# Functions : 함수
# ROUND(), TRUNCATE(), DATE_FORMAT(), CONCAT(), COUNT()

In [ ]:
QUERY = "USE world;"
curs.execute(QUERY)
QUERY = "select * from countrylanguage;"
df = pd.read_sql(QUERY, db)
df.head(3)

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7


In [ ]:
qr = '''
select countrycode, language, percentage, round(percentage, 0)
from countrylanguage;
'''
df=pd.read_sql(qr,db)
df.head()

,countrycode,language,percentage,"round(percentage, 0)"
0,ABW,Dutch,5.3,5.0
1,ABW,English,9.5,10.0
2,ABW,Papiamento,76.7,77.0
3,ABW,Spanish,7.4,7.0
4,AFG,Balochi,0.9,1.0


In [ ]:
QUERY = "SELECT ROUND(12.345, 2) AS number;"
df = pd.read_sql(QUERY,db)
df.head()

,number
0,12.35


In [ ]:
qr = 'select count(language) from countrylanguage;'
pd.read_sql(qr,db)

,count(language)
0,984


In [ ]:
qr = '''select count(distinct(language)) as uniq_lang
from countrylanguage;'''
pd.read_sql(qr, db)

,uniq_lang
0,457


In [ ]:
qr = "use sakila;"
curs.execute(qr)

0

In [ ]:
qr = "desc payment;"
pd.read_sql(qr, db)
qr = "select * from payment;"
an = pd.read_sql(qr, db)
an.head(3)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76.0,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573.0,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185.0,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30


In [ ]:
qr = '''
select amount, payment_date, last_update
,DATE_FORMAT(payment_date, "%M") as monthly
from payment;
'''
pd.read_sql(qr, db)

,amount,payment_date,last_update,monthly
0,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30,May
1,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30,May
2,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30,June
3,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30,June
4,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30,June
...,...,...,...,...
16044,4.99,2005-08-21 17:43:42,2006-02-15 22:24:12,August
16045,1.99,2005-08-21 21:41:57,2006-02-15 22:24:12,August
16046,8.99,2005-08-23 06:09:44,2006-02-15 22:24:12,August
16047,2.99,2005-08-23 11:08:46,2006-02-15 22:24:13,August


In [ ]:
qr = "desc staff;"
pd.read_sql(qr, db)
qr = '''
select staff_id, sum(amount) as amount
from payment
group by staff_id;
'''
an = pd.read_sql(qr, db)
an

,staff_id,amount
0,1,33489.47
1,2,33927.04


In [ ]:
QUERY = """
    SELECT amount, payment_date, last_update
           , DATE_FORMAT(payment_date, "%Y-%m") as monthly
    FROM payment;
"""
df = pd.read_sql(QUERY, db)
df.head(3)

,amount,payment_date,last_update,monthly
0,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30,2005-05
1,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30,2005-05
2,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30,2005-06


In [ ]:
# CONCAT() : 컬럼이나 데이터를 결합할때 사용
QUERY = """
    SELECT staff_id, first_name, last_name,
    concat(first_name, " ", last_name) as full_name
    FROM staff;
"""
df = pd.read_sql(QUERY, db)
df.head(3)

,staff_id,first_name,last_name,full_name
0,1,Mike,Hillyer,Mike Hillyer
1,2,Jon,Stephens,Jon Stephens


In [ ]:
qr = "use world;"
curs.execute(qr)
# 대륙별 전체인구를 구하고 5억이상인 대륙만 조회
qr = '''SELECT continent, SUM(Population) as Population
FROM country
GROUP BY continent
HAVING Population > 500000000;'''
an = pd.read_sql(qr, db)
an

,continent,Population
0,Asia,3.705026e+09
1,Africa,7.844750e+08
2,Europe,7.300746e+08


In [ ]:
qr = "use sakila;"
curs.execute(qr)
qr = "desc staff;"
pd.read_sql(qr, db)
qr = '''
select staff_id, sum(amount) as amount
from payment
group by staff_id;
'''
an = pd.read_sql(qr, db)
an

,staff_id,amount
0,1,33489.47
1,2,33927.04


In [ ]:
# payment 테이블에서 월별 총 매출 데이터 출력
# DATE_FORMAT(column, "%Y-%m")
# GROUP BY 절에 DATE_FORMAT 함수, 컬럼명 사용 가능
QUERY = """
    SELECT DATE_FORMAT(payment_date, "%Y-%m") as monthly
           , sum(amount) as total_sales
    FROM payment
    GROUP BY monthly
    ORDER BY total_sales DESC;
"""
df = pd.read_sql(QUERY, db)
df.head(10)

,monthly,total_sales
0,2005-07,28373.89
1,2005-08,24072.13
2,2005-06,9631.88
3,2005-05,4824.43
4,2006-02,514.18


In [ ]:
qr = "use world;"
curs.execute(qr)
# 대륙별 전체인구를 구하고 5억이상인 대륙만 조회
# HAVING : 결과 데이터에서 조건을 추가하여 데이터 출력
qr = '''SELECT continent, SUM(Population) as Population
FROM country
GROUP BY continent
HAVING Population > 50000*10000;'''
an = pd.read_sql(qr, db)
an

,continent,Population
0,Asia,3.705026e+09
1,Africa,7.844750e+08
2,Europe,7.300746e+08


In [ ]:
# where과 비교
# query가 실행될 떄 순서가 있다.
qr = '''SELECT continent, SUM(Population) as Population
FROM country
where Population > 50000*10000
GROUP BY continent;'''
an = pd.read_sql(qr, db)
an

,continent,Population
0,Asia,2.291220e+09


In [ ]:
# 대륙별 평균 인구수, 평균 GNP, 1인당 GNP한 결과를 1인당 GNP가 0.01 이상인 데이터를 조회하고 1인당 GNP를 내림차순으로 정렬
qr = '''
select continent, avg(population) as population_avg, avg(gnp) as gnp_avg, (AVG(GNP) / AVG(Population) * 1000) as AVG
from country
where GNP != 0 AND Population != 0
GROUP BY continent
HAVING AVG > 0.01
ORDER BY AVG DESC;
'''
an = pd.read_sql(qr, db)
an

,continent,population_avg,gnp_avg,AVG
0,North America,1.420109e+07,284959.623529,20.066041
1,Oceania,1.594684e+06,22093.405263,13.854408
2,Europe,1.659155e+07,215883.295455,13.011638
3,South America,2.659831e+07,116298.000000,4.372383
4,Asia,7.408281e+07,153107.840000,2.066712
5,Africa,1.426036e+07,10552.272727,0.739972


In [ ]:
qr = '''
select continent, region, sum(population) as total_population
from country
group by continent, region
with rollup;
'''
an = pd.read_sql(qr, db)
an.head(10)

,continent,region,total_population
0,Asia,Eastern Asia,1.507328e+09
1,Asia,Middle East,1.883807e+08
2,Asia,Southeast Asia,5.185410e+08
3,Asia,Southern and Central Asia,1.490776e+09
4,Asia,None,3.705026e+09
5,Europe,Baltic Countries,7.561900e+06
6,Europe,British Islands,6.339850e+07
7,Europe,Eastern Europe,3.070260e+08
8,Europe,Nordic Countries,2.416640e+07
9,Europe,Southern Europe,1.446742e+08


In [ ]:
QUERY = """
    create database kyobo;
    use kyobo;
"""
curs.execute(QUERY)

1

In [ ]:
# qr = 'drop table user;'
# curs.execute(qr)

In [ ]:
# qr = 'drop table addr;'
# curs.execute(qr)

In [ ]:
QUERY = """
    create table user(
        user_id int primary key auto_increment,
        name varchar(20)
    );
    create table addr(
        user_id int,
        addr_name varchar(20)
    );
"""
curs.execute(QUERY)

0

In [ ]:
qr = "show tables;"
pd.read_sql(qr,db)

,Tables_in_kyobo
0,addr
1,user


In [ ]:
# 테이블에 데이터 넣기
QUERY = """
    insert into user(name) values ("A"), ("B"), ("C");
"""
curs.execute(QUERY)

3

In [ ]:
QUERY = """
    insert into addr(user_id, addr_name) 
    values (1, "seoul"), (2, "pusan"), (4, "seoul"), (5, "inchone");
"""
curs.execute(QUERY)

4

In [ ]:
QUERY = """
    SELECT *
    FROM user
    JOIN addr;
"""
df = pd.read_sql(QUERY, db)
df.head(20)

,user_id,name,user_id,addr_name
0,3,C,1,seoul
1,2,B,1,seoul
2,1,A,1,seoul
3,3,C,2,pusan
4,2,B,2,pusan
5,1,A,2,pusan
6,3,C,4,seoul
7,2,B,4,seoul
8,1,A,4,seoul
9,3,C,5,inchone


In [ ]:
# INNER JOIN : INNER 생략 가능
QUERY = """
    SELECT user.user_id, user.name, addr.addr_name
    FROM user
    JOIN addr
    ON user.user_id = addr.user_id;
"""
df = pd.read_sql(QUERY, db)
df.head(20)

,user_id,name,addr_name
0,1,A,seoul
1,2,B,pusan


In [ ]:
qr = 'use world;'
curs.execute(qr)
qr = 'show tables;'
pd.read_sql(qr,db)

,Tables_in_world
0,addr
1,city
2,city900
3,city_900
4,country
5,countrylanguage
6,user


In [ ]:
# Q : 국가코드, 국가이름, 도시이름, 국가인구, 도시인구를 출력
qr = 'desc country;'
pd.read_sql(qr,db)

,Field,Type,Null,Key,Default,Extra
0,Code,char(3),NO,PRI,,
1,Name,char(52),NO,,,
2,Continent,"enum('Asia','Europe','North America','Africa',...",NO,,Asia,
3,Region,char(26),NO,,,
4,SurfaceArea,"decimal(10,2)",NO,,0.00,
5,IndepYear,smallint,YES,,None,
6,Population,int,NO,,0,
7,LifeExpectancy,"decimal(3,1)",YES,,None,
8,GNP,"decimal(10,2)",YES,,None,
9,GNPOld,"decimal(10,2)",YES,,None,


In [ ]:
qr = 'desc city;'
pd.read_sql(qr,db)

,Field,Type,Null,Key,Default,Extra
0,ID,int,NO,PRI,None,auto_increment
1,Name,char(35),NO,,,
2,CountryCode,char(3),NO,MUL,,
3,District,char(20),NO,,,
4,Population,int,NO,,0,


In [ ]:
qr = '''
select country.code, country.name as country_name, 
city.name as city_name, country.population as country_pop, city.population as city_pop
from country
join city
on country.code = city.countrycode
'''
df = pd.read_sql(qr,db)
df.head(10)

,code,country_name,city_name,country_pop,city_pop
0,ABW,Aruba,Oranjestad,103000,29034
1,AFG,Afghanistan,Kabul,22720000,1780000
2,AFG,Afghanistan,Qandahar,22720000,237500
3,AFG,Afghanistan,Herat,22720000,186800
4,AFG,Afghanistan,Mazar-e-Sharif,22720000,127800
5,AGO,Angola,Luanda,12878000,2022000
6,AGO,Angola,Huambo,12878000,163100
7,AGO,Angola,Lobito,12878000,130000
8,AGO,Angola,Benguela,12878000,128300
9,AGO,Angola,Namibe,12878000,118200


In [ ]:
# Q : 국가인구수에 대한 도시 인구수의 비율 컬럼 추가, 도시인구의 비율이 20% 넘는 데이터를 출력
qr = '''
select country.code, country.name as country_name, 
city.name as city_name, country.population as country_pop, city.population as city_pop,
concat(round(city.population / country.population, 2)*100, "%") as percent_pop
from country
join city
on country.code = city.countrycode
having percent_pop > 20;
'''
df = pd.read_sql(qr,db)
df.head(10)

,code,country_name,city_name,country_pop,city_pop,percent_pop
0,ABW,Aruba,Oranjestad,103000,29034,28.00%
1,AND,Andorra,Andorra la Vella,78000,21189,27.00%
2,ARE,United Arab Emirates,Dubai,2441000,669181,27.00%
3,ARM,Armenia,Yerevan,3520000,1248700,35.00%
4,ATG,Antigua and Barbuda,Saint John´s,68000,24000,35.00%
5,AZE,Azerbaijan,Baku,7734000,1787800,23.00%
6,BHR,Bahrain,al-Manama,617000,148000,24.00%
7,BHS,Bahamas,Nassau,307000,172000,56.00%
8,BLZ,Belize,Belize City,241000,55810,23.00%
9,CCK,Cocos (Keeling) Islands,Bantam,600,503,84.00%


In [ ]:
QUERY = """
    use kyobo;
"""
curs.execute(QUERY)

0

In [ ]:
QUERY = """
    SELECT user.user_id, user.name, addr.addr_name
    FROM user
    LEFT JOIN addr
    ON user.user_id = addr.user_id
    UNION
    SELECT addr.user_id, user.name, addr.addr_name
    FROM user
    RIGHT JOIN addr
    ON user.user_id = addr.user_id;
"""
df = pd.read_sql(QUERY, db)
df

,user_id,name,addr_name
0,1,A,seoul
1,2,B,pusan
2,3,C,None
3,4,None,seoul
4,5,None,inchone


In [ ]:
qr = "use world;"
curs.execute(qr)

0

In [ ]:
# SELECT : 1개의 row에 전체국가수, 전체도시수, 전체언어수
QUERY = """
    SELECT COUNT(*) FROM country;
"""
QUERY = """
    SELECT COUNT(*) FROM city;
"""
QUERY = """
    SELECT COUNT(DISTINCT(language)) FROM countrylanguage;
"""
pd.read_sql(QUERY, db)

,COUNT(DISTINCT(language))
0,457


In [ ]:
# SELECT : 1개의 row에 전체국가수, 전체도시수, 전체언어수
QUERY = """
    SELECT (SELECT COUNT(*) FROM country) AS total_country
           , (SELECT COUNT(*) FROM city) AS total_city
           , (SELECT COUNT(DISTINCT(language)) FROM countrylanguage)
           AS total_language
"""
pd.read_sql(QUERY, db)

,total_country,total_city,total_language
0,239,4079,457


In [ ]:
# FROM :  800만 이상이 되는 도시의 국가코드, 국가이름, 도시이름, 도시인구수 출력
# 1) having 사용
qr = """
select country.code, country.name, city.name, city.population
from city
join country on country.code = city.countrycode
having city.population >= 900*10000
"""
pd.read_sql(qr,db)

,code,name,name,population
0,BRA,Brazil,São Paulo,9968485
1,CHN,China,Shanghai,9696300
2,IDN,Indonesia,Jakarta,9604900
3,IND,India,Mumbai (Bombay),10500000
4,KOR,South Korea,Seoul,9981619
5,PAK,Pakistan,Karachi,9269265


In [ ]:
# 2) sub-query 사용
qr = """
select country.code, country.name, city.name, city.population
from (select * from city where population >= 900*10000) as city
join country on country.code = city.countrycode
"""
pd.read_sql(qr,db)
# 가져오는 수가 더 적어지니까 subquery가 더 좋겠징? 6 * 239 = 1,434

,code,name,Name,Population
0,BRA,Brazil,São Paulo,9968485
1,IDN,Indonesia,Jakarta,9604900
2,IND,India,Mumbai (Bombay),10500000
3,CHN,China,Shanghai,9696300
4,KOR,South Korea,Seoul,9981619
5,PAK,Pakistan,Karachi,9269265


In [ ]:
# WHERE안에 sub-query : 도시인구 900만이상의 도시의 국가코드, 국가이름, 대통령이름 출력
QUERY = """
    SELECT countrycode
    FROM city 
    WHERE population >= 900*10000
"""
QUERY = """
    SELECT code, name, headofstate
    FROM country
    WHERE code IN ('BRA', 'IDN', 'IND')
"""
QUERY = """
    SELECT code, name, headofstate
    FROM country
    WHERE code IN (
        SELECT countrycode
        FROM city 
        WHERE population >= 900*10000
    )
"""
pd.read_sql(QUERY, db)

,code,name,headofstate
0,BRA,Brazil,Fernando Henrique Cardoso
1,CHN,China,Jiang Zemin
2,IDN,Indonesia,Abdurrahman Wahid
3,IND,India,Kocheril Raman Narayanan
4,KOR,South Korea,Kim Dae-jung
5,PAK,Pakistan,Mohammad Rafiq Tarar


In [ ]:
# VIEW : 가상의 테이블 : 데이터의 수정, 삭제, 추가등이 불가
# 쿼리의 길이를 줄여줌

QUERY = """
    SELECT city.countrycode, country.name, city.name, city.population
    FROM (SELECT * FROM city WHERE population >= 900*10000) AS city
    JOIN country
    ON city.countrycode = country.code
"""
pd.read_sql(QUERY, db)

,CountryCode,name,Name,Population
0,BRA,Brazil,São Paulo,9968485
1,IDN,Indonesia,Jakarta,9604900
2,IND,India,Mumbai (Bombay),10500000
3,CHN,China,Shanghai,9696300
4,KOR,South Korea,Seoul,9981619
5,PAK,Pakistan,Karachi,9269265


In [ ]:
# view 생성
QUERY = """
    CREATE VIEW cityy900 AS
    SELECT * FROM city WHERE population >= 900*10000;
"""
curs.execute(QUERY)

0

In [ ]:
# view 확인
QUERY = """
    SHOW FULL TABLES WHERE Table_type = "VIEW"
"""
pd.read_sql(QUERY, db)

,Tables_in_world,Table_type
0,city900,VIEW
1,city_900,VIEW
2,cityy900,VIEW


In [ ]:
QUERY = """
    SELECT city.countrycode, country.name, city.name, city.population
    FROM cityy900 AS city
    JOIN country
    ON city.countrycode = country.code
"""
pd.read_sql(QUERY, db)

,CountryCode,name,Name,Population
0,BRA,Brazil,São Paulo,9968485
1,IDN,Indonesia,Jakarta,9604900
2,IND,India,Mumbai (Bombay),10500000
3,CHN,China,Shanghai,9696300
4,KOR,South Korea,Seoul,9981619
5,PAK,Pakistan,Karachi,9269265


In [ ]:
# 인덱스 확인
qr = '''
show index from city
'''
pd.read_sql(qr,db)

,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,city,0,PRIMARY,1,city_id,A,600,None,None,,BTREE,,,YES,None
1,city,1,idx_fk_country_id,1,country_id,A,109,None,None,,BTREE,,,YES,None


In [ ]:
qr = '''
select*from city;
'''
an = pd.read_sql(qr,db)
an.head(3)

,city_id,city,country_id,last_update
0,1,A Corua (La Corua),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25
2,3,Abu Dhabi,101,2006-02-15 04:45:25


In [ ]:
# 데이터 검색 속도 확인
%%time
qr = '''
select * from city
where city = "Seoul";
'''
df = pd.read_sql(qr,db)

CPU times: user 4.44 ms, sys: 847 µs, total: 5.28 ms
Wall time: 234 ms
